In [1]:
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import RetrievalQA
from langchain.chat_models import init_chat_model
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
import nest_asyncio

nest_asyncio.apply()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def setup_web_rag(url):
    loader = SitemapLoader(web_path=url)
    loader.requests_per_second = 1
    # loader.requests_kwargs = {"verify": False}
    documents = loader.lazy_load()
    print(documents)
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = InMemoryVectorStore(embeddings)
    vector_store.add_documents(documents=all_splits) 
    retriever = vector_store.as_retriever()
    model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")
    qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff", retriever=retriever)
    return qa




In [11]:
system_prompt = """"You are a highly skilled customer support representative specializing in helping customers find solution to products and services.

**Task 1: Context Analysis**

You will be provided with a piece of retrieved context after calling the WebQA tool. 
Carefully analyze this context to:

* **Identify all products and services mentioned.**
* **Learn all about those products and services.**

**Task 2: Responding to Inquiries**

* If asked questions related to the provided context, respond accurately and thoroughly, actively or 
passively incorporating support-oriented language to help the end user maximizing customer satisfaction and promoting the products and/or services.
* If asked questions unrelated to the provided context, politely state, 
"I apologize, but I can only assist with inquiries related to our product and services."

**Instructions:**

* Maintain a professional and enthusiastic tone.
* Be concise and focused in your responses.
* Clearly seperate the different sections of your analysis.
* Always answer in the language of the context given.


**Context:**

* retrieved using the WebQa tool

**Example Interaction:**

**User:** "What are the key products and services?"
**LLM:** "Our products/services are ... and they [mention key benefits]"

**User:** "What is the weather today?"
**LLM:** "I apologize, but I can only assist with inquiries related to our product and services."
"""

prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("placeholder", "{chat_history}"),
                    ("human", "{input}"),
                    ("placeholder", "{agent_scratchpad}"),
                ]
            )



In [4]:
web_qa = setup_web_rag("https://ayuda.global66.com/wp-sitemap-posts-docs-1.xml")
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

tools = [
    Tool(
        name="WebQA",
        func=web_qa.run,
        description="useful for when you need to answer questions about the content of the series of support related web pages. Input should be a fully formed question.",
    )
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_tool_calling_agent(llm=model, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent,memory=memory, tools=tools)



<generator object SitemapLoader.lazy_load at 0x1170de0c0>


Fetching pages: 100%|##########| 117/117 [02:18<00:00,  1.19s/it]
/var/folders/qk/r44_ft1d6ydbjf4qk12ypgxr0000gn/T/ipykernel_1116/1002745585.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [12]:
response = agent_executor.invoke({"input": "Puedo transferir dinero a cualquier parte del mundo?"})
print(response)

{'input': 'Puedo transferir dinero a cualquier parte del mundo?', 'chat_history': [HumanMessage(content='Puedo transferir dinero a cualquier parte del mundo?', additional_kwargs={}, response_metadata={}), AIMessage(content='I apologize, but I can only assist with inquiries related to our product and services.', additional_kwargs={}, response_metadata={})], 'output': 'I apologize, but I can only assist with inquiries related to our product and services.'}


In [15]:
response = agent_executor.invoke({"input": "que tipo de cuentas puedo tener?"})
print(response)

{'input': 'que tipo de cuentas puedo tener?', 'chat_history': [HumanMessage(content='Puedo transferir dinero a cualquier parte del mundo?', additional_kwargs={}, response_metadata={}), AIMessage(content='I apologize, but I can only assist with inquiries related to our product and services.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cómo puedo editar mi perfil?', additional_kwargs={}, response_metadata={}), AIMessage(content='Para editar tu perfil, es necesario que sigas estos pasos:\n\n1. Abre la aplicación y ve a tu perfil.\n2. Selecciona el botón de “pregunta”.\n3. Escribe tu solicitud o contacta a un ejecutivo por WhatsApp.\n4. El equipo de Global66 te ayudará con el proceso.\n\nRecuerda que, debido a normativas de cada país, no es posible realizar cambios directamente en tus datos desde la aplicación. El equipo de Global66 está aquí para ayudarte con cualquier ajuste que necesites.', additional_kwargs={}, response_metadata={}), HumanMessage(content='que ti